In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil

import tensorflow as tf
from keras.layers import Dropout

from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Flatten, Input, concatenate
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import tensorflow.keras.utils as utils
from keras.callbacks import EarlyStopping

from datetime import datetime

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from google.colab import drive
import zipfile

In [ ]:
train_data = "/content/drive/MyDrive/train"
test_data = "/content/drive/MyDrive/test"

input_shape = (64,64)

train_gen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_gen = ImageDataGenerator(rescale=1./255)

train_dataset = train_gen.flow_from_directory(train_data, target_size=input_shape, batch_size=32, class_mode='categorical')
test_dataset = test_gen.flow_from_directory(test_data, target_size=input_shape, batch_size=32, class_mode='categorical')

In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D

input_shape = (224,224)

base_model = MobileNet(input_shape=(input_shape[0], input_shape[1], 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(train_dataset.num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

result = model.fit(train_dataset, epochs=50, validation_data=test_dataset, batch_size=32)

test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


# Early Stopping

# early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# result = model.fit(train_dataset, epochs=50, validation_data=test_dataset, batch_size=32, callbacks=[early_stopping])

# test_loss, test_accuracy = model.evaluate(test_dataset)

# print(f'Test Loss: {test_loss}')
# print(f'Test Accuracy: {test_accuracy}')



In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)

plt.title('LOSS')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.grid()

plt.plot(result.history['loss'], label='train_loss')
plt.plot(result.history['val_loss'], label= 'validation_loss')
plt.legend(loc='best')

plt.subplot(1,2,2)

plt.title('ACCURACY')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.grid()

plt.plot(result.history['accuracy'], label='train_accuracy')
plt.plot(result.history['val_accuracy'], label= 'validationi_accuracy')
plt.legend(loc='best')